# Friedel Pair Mapping notebook

The idea behind this is similar to the idea in DCT and here https://doi.org/10.1107/S1600576724009634, where Friedel pairs are used to locate where diffraction spots come from in space. In those cases we use peaks that are 180 degrees apart. This notebook is looking for peaks that are separated by twotheta. These are the peaks we use in the friedel_rois macro at ID11 that aligns grains on the centre of rotation.

The pairs we use will have:
- eta -> -eta
- tth -> tth
- gve -> -gve

Jon Wright. March 2025.


In [ ]:
dset_file ="/path/to/dataset"
phase_name = 'phase' # or None
y0 = 0.0
gvtol = 0.002     # value is often OK
# xpos, ypos and ytol erance for a position in space.
# update these after you see your plot
    
ytol = 1.0        # For selecting peaks in space
px = 0.0    
py = 0.0


# test dataset:
if False:
    dset_file = "/data/id11/inhouse2/test_data_3DXRD/S3DXRD/FeAu/PROCESSED_DATA/20250303_JW/tomo_route/FeAu_0p5_tR_nscope/FeAu_0p5_tR_nscope_top_200um/FeAu_0p5_tR_nscope_top_200um_dataset.h5"
    y0 = -15.9        # matches your dataset, if known.
    phase_name = 'Fe' 
    # xpos, ypos and tolerance for a position in space.
    # update these after you see your plot
    px = -108.
    py = -164.
    ytol = 2.

In [ ]:
import os, sys, time
start = time.time()
# USER: You can change this location if you want
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git()

In [ ]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
import ImageD11.sinograms.dataset
import ImageD11.sinograms.geometry
import ImageD11.transformer
import ImageD11.indexing
import ImageD11.sinograms.roi_iradon
import scipy.spatial

In [ ]:
ds = ImageD11.sinograms.dataset.load(dset_file)
print(ds)

In [ ]:
cf_4d = ds.get_cf_4d()
ds.update_colfile_pars(cf_4d, phase_name)
print(cf_4d.nrows/1e6, "million peaks read in")

In [ ]:
def find_pairs_minus_eta( cf, gvtol = 0.002 ):
    """
    Locate Friedel pairs with eta -> -eta and g -> -g
    returns ip, im  == indices for eta+ and eta- pairs
    """
    # select peaks from left or right of detector
    fp = np.mgrid[0:cf.nrows][cf_4d.eta > 0 ]
    fm = np.mgrid[0:cf.nrows][cf_4d.eta < 0 ]
    # gvector arrays of these peaks,  make into KD trees
    kdp = scipy.spatial.cKDTree(  np.transpose( (cf.gx[fp], cf.gy[fp], cf.gz[fp]) ) )
    kdm = scipy.spatial.cKDTree( -np.transpose( (cf.gx[fm], cf.gy[fm], cf.gz[fm]) ) )
    # Find the pairs
    coo = kdp.sparse_distance_matrix( kdm, gvtol, output_type = 'coo_matrix' )
    # Return the indices in the original cf_4d
    return fp[coo.row], fm[coo.col]

def locate_pairs( cf, pairs, y0 = 0. ):
    """
    Fit the centre of mass position of the pairs
    cf = colfile
    pairs = (ip, im) = indices of low, high pair in cf
    
    Returns sx, sy == sample x and y co-ordinates of the peak-pair
    """
    ip, im = pairs
    r = np.radians(cf.omega )
    so = np.sin(r)
    co = np.cos(r)
    # For each paired peak take dty - y0 == observed dty value
    y = np.transpose((cf.dty[ip]-y0, cf.dty[im]-y0 ))
    # Find the 2x2 matrix for fitting the dty position (-,-) in geometry notebook
    R = np.transpose( (( -so[ip], -co[ip] ),
                       ( -so[im], -co[im] )), axes=(2,0,1))
    # Solve for x,y in the sample co-ordinates
    return np.linalg.solve( R, y ).T

The next cell is locating the Friedel pairs. It seems to need about 1 second per million peaks.

In [ ]:
ip, im = find_pairs_minus_eta( cf_4d, gvtol=gvtol )
print('Got',len(ip),'pairs from',cf_4d.nrows,'peaks, fraction paired =',len(ip)*2/cf_4d.nrows )

Now fit the positions. Should be faster than finding the pairs.

In [ ]:
sx, sy = locate_pairs( cf_4d, (ip,im), y0 = y0 )

For the plots, we have selected a position in space to extract a grain:

In [ ]:
# Mask for this position in space
m = ((abs(sx-px) < ytol) & (abs(sy-py) < ytol))
idxpt = np.concatenate( (ip[m], im[m]))
# gvectors from the point px,py in the sample
gvp =  (cf_4d.gx[idxpt],cf_4d.gy[idxpt],cf_4d.gz[idxpt]) 

# verify whether we got the geometry right
xfit,yfit,y0fit = ImageD11.sinograms.geometry.fit_sine_wave(cf_4d.omega[idxpt], cf_4d.dty[idxpt], [0.1,0.1,y0])
calcy = ImageD11.sinograms.geometry.dtycalc(ds.obincens, xfit, yfit, y0fit )

In [ ]:
f = plt.figure(figsize=(10,4),constrained_layout=True)
a = [f.add_subplot(131),f.add_subplot(132),f.add_subplot(133, projection='3d', proj_type='ortho') ]
f.colorbar( a[0].hist2d( sx, sy, bins=ds.ybinedges, norm='log', zorder=1)[-1] )
a[0].scatter(px,py,s=10,color='k',fc='none',ec='k')
a[0].set(aspect='equal', xlabel='x sample' , ylabel='ysample',title='Pair locations')
a[1].plot( cf_4d.omega[idxpt], cf_4d.dty[idxpt], '.')
a[1].plot( ds.obincens, calcy, '-', label='fitted' )
a[1].set(title='Fit: x %.3f, y %.3f y0 %.3f'%( xfit, yfit, y0fit ), xlabel='omega', ylabel='dty');
# Select peaks from some position in space
a[2].scatter( *gvp,',',s=1)
a[2].set(title=f"Selected gve",
      xlabel='gx', ylabel='gy', zlabel='gz');

## Check: run some indexing

In [ ]:
def run_index_unknown(gid, cf, frac=0.2, tol=0.05, sigma=5):
    """
    gid = string to name files
    cf = colfile to index
    frac = fraction of peaks you want to index
    tol = hkl tolerance
    """
    tr = ImageD11.transformer.transformer()
    tr.colfile = cf
    tr.parameterobj = cf.parameters
    # need to have cell params to save gves
    tr.parameterobj.set('cell_lattice_[P,A,B,C,I,F,R]','P')# integer not backwards compatible
    tr.savegv( f'gr{gid}.gve' )
    !index_unknown.py -g gr{gid}.gve -m 40 --fft -t {tol} -f {frac} -o {gid}.ubi -k 1 -s {sigma}
    if os.path.exists(f'{gid}.ubi'):
        fixhandedness( f'{gid}.ubi' ) # the script on path might not be the one in git
    
def fixhandedness( ubifile ):
    ubis = ImageD11.indexing.readubis( ubifile )
    for i in range(len(ubis)):
        if np.linalg.det( ubis[i] ) < 0:
            ubis[i][-1] = -ubis[i][-1]
        assert np.linalg.det( ubis[i] ) > 0
    ImageD11.indexing.write_ubi_file(  ubifile, ubis )

In [ ]:
run_index_unknown( '0', cf_4d.copyrows( idxpt ), frac=0.2, tol=0.1, sigma=10)

You can check for higher symmetry at https://www.cryst.ehu.es/cryst/lattice.html

In [ ]:
print('Total runtime', time.time()-start)